# Title, no params

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import altair as alt
import geopandas as gpd
import intake
import pandas as pd

from IPython.display import display, Markdown, HTML

import parallel_corridors_utils
from shared_utils import styleguide

catalog = intake.open_catalog("./*.yml")
alt.themes.register("calitp_theme", styleguide.calitp_theme)
alt.renderers.enable("html")

In [ ]:
# parameters cell
itp_id = 182

In [ ]:
df = catalog.competitive_route_variability.read()

df = (df[(df.calitp_itp_id == itp_id) & 
         (df.plot_group.notna())]
      .reset_index(drop=True)
     )


if itp_id == 182:
    df = df.assign(
        route_id = df.route_id.str.replace('-13153', '').astype(int)
    )

In [ ]:
# Grab parameters for narrative text - clean up formatting here

# Set up df for charting (cut-off at some threshold to show most competitive routes)
PCT_COMPETITIVE_THRESHOLD = 0.75
plot_me = df[df.pct_trips_competitive > PCT_COMPETITIVE_THRESHOLD].drop(columns = "geometry")
    
operator_name = df.name.iloc[0]
district = df.caltrans_district.iloc[0]
formatted_date = pd.to_datetime(parallel_corridors_utils.SELECTED_DATE).strftime('%m-%d-%Y')


stats = parallel_corridors_utils.operator_parallel_competitive_stats(
    itp_id, PCT_COMPETITIVE_THRESHOLD)

pct_parallel = round(stats['parallel_routes'] / stats['num_routes'] * 100,1 )
pct_competitive =  round(stats['competitive_routes'] / stats['num_routes'] * 100,1 )

In [ ]:
display(Markdown("# Competitive Route Travel Time Variability"))
display(HTML(f"<h1>{operator_name} (ITP ID: {itp_id})</h1>"))

In [ ]:
display(
    HTML(f"<h4>{operator_name} Date: {formatted_date}</h4>")
)
display(
    Markdown(
        f"**Bus routes in service**: {stats['num_routes']} "
        "<br>**Parallel routes** to State Highway Network (SHN): "
        f"{stats['parallel_routes']} routes ({pct_parallel}%)"
        f"<br>**Competitive routes** against car travel: {stats['competitive_routes']} routes ({pct_competitive}%)"
        "<br>"
        "Notes: <br>*Parallel routes* are routes where at least 30% of the route falls "
        " within 1 mile of the SHN "
        "and cover at least 10% of the highway's length."
        "<br>"
        "<br>*Competitive routes* are routes where at least "
        f"{int(PCT_COMPETITIVE_THRESHOLD*100)}% of the trips on that route take "
        "no more than 2x the car would. The fastest trip for the bus route is "
        "selected to estimate against the car traveling along the same path "
        "at the same departure hour on the same day. A ratio of bus travel time "
        "to car travel time is taken for all other trips using that car travel time, "
        "and a percentage of all trips that fall within the no-more-than-twice-the-car-travel threshold is calculated for the route."
    )
)

In [ ]:
y_col1 = "bus_multiplier"
Y_MIN1, Y_MAX1 = parallel_corridors_utils.set_yaxis_range(plot_me, y_col1)

y_col2 = "bus_difference"
Y_MIN2, Y_MAX2 = parallel_corridors_utils.set_yaxis_range(plot_me, y_col2)

groups = list(sorted(plot_me.plot_group.unique()))

for i in groups:
    multiplier_chart = parallel_corridors_utils.make_stripplot(
        plot_me[plot_me.plot_group==i], y_col1, Y_MIN = Y_MIN1, Y_MAX = Y_MAX1
    )
    
    difference_chart = parallel_corridors_utils.make_stripplot(
        plot_me[plot_me.plot_group==i], y_col2, Y_MIN = Y_MIN2, Y_MAX = Y_MAX2
    )

    display(multiplier_chart)
    display(difference_chart)